<a href="https://colab.research.google.com/github/mbuguajnr23/Deep_Learning/blob/main/Multithreaded_Image_Downloader_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import requests
import threading
from queue import Queue
from urllib.parse import urlparse

In [8]:
# Configuration
DOWNLOAD_DIR = "/content/downloaded_images"  # Colab-specific directory
NUM_THREADS = 5  # Number of threads to use for downloading

# Create the download directory if it doesn't exist
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)

def download_image(url, queue):
    """
    Downloads an image from the given URL and saves it to the local directory.
    """
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Extract the filename from the URL
        parsed_url = urlparse(url)
        filename = os.path.basename(parsed_url.path)
        if not filename:
            filename = f"image_{queue.qsize()}.jpg"  # Default filename if URL doesn't contain one

        # Save the image to the local directory
        filepath = os.path.join(DOWNLOAD_DIR, filename)
        with open(filepath, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"Downloaded: {filename}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")
    finally:
        queue.task_done()  # Mark the task as done

In [9]:
def worker(queue):
    """
    Worker function that processes URLs from the queue.
    """
    while True:
        try:
            url = queue.get_nowait()  # Get a URL from the queue
        except:
            break  # Exit if the queue is empty
        download_image(url, queue)

In [10]:
def get_user_urls():
    """
    Prompts the user to input image URLs one by one.
    """
    urls = []
    print("Enter the image URLs one by one. Type 'done' when finished:")
    while True:
        url = input("Enter URL: ").strip()
        if url.lower() == "done":
            break
        if url:  # Ensure the input is not empty
            urls.append(url)
    return urls

In [11]:
def main(urls):
    """
    Main function to manage the download process.
    """
    # Create a queue and add all URLs to it
    queue = Queue()
    for url in urls:
        queue.put(url)

    # Create and start threads
    threads = []
    for _ in range(NUM_THREADS):
        thread = threading.Thread(target=worker, args=(queue,))
        thread.start()
        threads.append(thread)

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    print("All downloads completed.")

In [12]:
if __name__ == "__main__":
    # Get image URLs from user input
    image_urls = get_user_urls()

    # Start the download process
    if image_urls:
        main(image_urls)
        # List files in the download directory
        !ls /content/downloaded_images
    else:
        print("No URLs provided. Exiting.")

Enter the image URLs one by one. Type 'done' when finished:
Enter URL: https://i.pinimg.com/736x/92/16/e8/9216e891fb4cb48deaf1d6b9f73c0353.jpg
Enter URL: https://i.pinimg.com/736x/a3/88/32/a38832da298d079ac0cd2b98686e5201.jpg
Enter URL: https://i.pinimg.com/736x/b7/56/f3/b756f38be3f7fb7c8d711ef77c16c822.jpg
Enter URL: https://i.pinimg.com/736x/2d/8c/af/2d8cafc885db059db25aa9c7cbeef36b.jpg
Enter URL: 
Enter URL: done
Downloaded: a38832da298d079ac0cd2b98686e5201.jpg
Downloaded: 9216e891fb4cb48deaf1d6b9f73c0353.jpg
Downloaded: 2d8cafc885db059db25aa9c7cbeef36b.jpg
Downloaded: b756f38be3f7fb7c8d711ef77c16c822.jpg
All downloads completed.
2d8cafc885db059db25aa9c7cbeef36b.jpg  a38832da298d079ac0cd2b98686e5201.jpg
51ce6a3f68585bb4bf3963a09ea7c323.jpg  b756f38be3f7fb7c8d711ef77c16c822.jpg
9216e891fb4cb48deaf1d6b9f73c0353.jpg
